In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'e:/Study/GraduationProject/Covid/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    'e:/Study/GraduationProject/Covid/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy',Recall()])

model.fit(train_data,
          validation_data=test_data,
          epochs=10)

loss, acc,recall = model.evaluate(test_data)
print(f"Test Accuracy: {acc * 100:.2f}%")

model.save("covid_model.h5")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

img_path = "e:/Study/GraduationProject/Covid/test/Normal/0108.jpeg"
img = image.load_img(img_path, target_size=(224, 224))

plt.imshow(img)
plt.axis('off')
plt.show()

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

prediction = model.predict(img_array)[0][0]

if prediction <= 0.5:
    print(f"COVID ({prediction:.2f})")
else:
    print(f"Normal ({1 - prediction:.2f})")


In [ ]:
loss, acc, recall = model.evaluate(test_data)
print(f"Test Accuracy: {acc * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")